## Validate ARGO R4 Resources from Servers

- Get Resources
- Load into R4 IG
- Run IG 
- Review QA report for Examples

(Python 3.7) 
**Run in MAC local source directory**

### Import modules and asign globals

In [1]:
import os #os module imported here
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from pathlib import Path
from pandas import *
from collections import namedtuple
import bearer as b

#Globals

# if working on Mac '/Users/ehaas/...'
# if working on PC '//ERICS-AIR-2/ehaas...'
source_path = '/ehaas/Documents/FHIR/ArgoR4Validator'
examples_path = '/ehaas/Documents/FHIR/ArgoR4Validator/source/examples'

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json',
    'Authorization': b.bearer,
    }

# US Core R4 Profiles
types = dict (
Patient = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient',
AllergyIntolerance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',
CarePlan = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan',
CareTeam = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam',
Condition = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition',
Device = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device',
DiagnosticReport = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab',
                    'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note'],
DocumentReference = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference',
Encounter = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter',
Goal = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal',
Immunization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization',
Location = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location',
Medication = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication',
MedicationRequest = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest',
Observation = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab',
               'http://hl7.org/fhir/StructureDefinition/vitalsigns',
                'http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus',
               'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height',
               'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age',
               'http://hl7.org/fhir/us/core/StructureDefinition/us-core-pulse-oximetry',],
Organization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',
Practitioner = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner',
PractitionerRole = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole',
Procedure = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure',
Provenance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-provenance',
)

f_servers = dict(epic='https://connectathon.epic.com/Interconnect-Fhir-Unsecure/api/FHIR/R4/',
                cerner='https://fhir-ehr.sandboxcerner.com/r4/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca/',
                )

### Optionally Clear example folder first...  (todo)

### Get examples from FHIR Server

- import data from spreadsheet
- convert to pandas df and access using dot notation


In [ ]:
server = 'cerner'  # sheet name = server
xls = ExcelFile('test_resources.xlsx', dtype=str)
df = read_excel(xls,server,na_false = False)

df

### Add meta profile to example

- for each resource type fetching instance
  - instance are comma separated list of ids
- convert json to dict
- insert profile
- update all references to relative references using same server ids
- convert back to json

- save as example in examples folder in ig publisher 
- create a df file and export as csv data file for mapping the file name to id for ig publisher

In [ ]:

def make_rel_ref(dictionary): # replace all absolute references with relative references
    for k, v in dictionary.items():
        if k == 'reference':    
            
            # make rel ref assuming is Type/id
            ref_path = dictionary[k].split('/')
            try:
                dictionary[k] = f'{ref_path[-2]}/{ref_path[-1]}'
            except IndexError:
                pass
        elif isinstance(v, dict):
            make_rel_ref(v)
        elif isinstance(v, list):
            for d in v:
                if isinstance(d, dict):
                    make_rel_ref(d)
    return(dictionary)

data_file = {'file_name': [],
        'id': [],
        }
for i in df.itertuples(index=True):
    for j in i.id.split(','):
        print(i.resource_type,j)
        params = dict(
            _id = j
            )
        # Open Server
        r = get(f'{f_servers[server]}{i.resource_type}/{j}', headers = headers)
        fhir_d = r.json() if r.status_code==200 else None
        #print(r.request.body)
        #for k,v in r.request.headers.items():
        #    print(f'{k} {v}')
        print(r.status_code)

        if r.status_code!=200:
            print(f'Fetching of {f_servers[server]}{i.resource_type}/{j} failed: Status code= {r.status_code}')
            r = get(f'{f_servers[server]}{i.resource_type}',params = params, headers = headers ) # try other syntax

            fhir_d = r.json()['entry'][0]['resource'] if r.status_code==200 else None #unbundle
            # wait for error to do try pattern
            print(r.status_code)
        if r.status_code!=200:
            print(f'Seaching using {f_servers[server]}{i.resource_type}?_id={j} failed: Status code= {r.status_code}')     
        else: # r.status_code==200
            try:
                fhir_d['meta']['profile'].append(profiles[i.resource_type])
            except KeyError:
                fhir_d['meta']= dict(profile = [profiles[i.resource_type]])
            fhir_d = make_rel_ref(fhir_d)  # make rel ref assuming is Type/id
            fhir_json = dumps(fhir_d, indent = 4)
            print(f'{"*"*10} {server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json {"*"*10}')
            print(fhir_json)
            try:
                p = Path(f'/Users{examples_path}') / f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json'
                #print(p)
                p.write_text(fhir_json, encoding='utf-8')
                print('writing to examples folder......')
            except FileNotFoundError: # try different path
                p = Path(f'//ERICS-AIR-2{examples_path}') / f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json'
                #print(p)
                p.write_text(fhir_json, encoding='utf-8')
                print('writing to examples folder......')        
            data_file['file_name'].append(f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}')
            data_file['id'].append(fhir_d["id"])
  

#### Save file name, id data as csv file

In [ ]:
df_data = DataFrame(data_file, columns= ['file_name', 'id'])
try:
    export_csv = df_data.to_csv(f'{source_path}/framework/_data/examples.csv', index = None, header=False, mode='a') #Don't forget to add '.csv' at the end of the path
except FileNotFoundError:
    export_csv = df_data.to_csv(f'//ERICS-AIR-2/{source_path}/framework/_data/examples.csv', index = None, header=False, mode='a') #Don't forget to add '.csv' at the end of the path    
print('saving csv .......')

## **** From Here on it only works on local machine ( MAC ) ****

### Validate using IG Build

1. run py script to update ig.json and ig.xml

In [ ]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/ArgoR4Validator/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
source_path=/Users/ehaas/Documents/FHIR/ArgoR4Validator
path3=/Users/ehaas/Documents/FHIR/IG-tools/
echo "================================================================="
echo === use definition files from relative path ../$SOURCE ===
echo "================================================================="
echo getting rid of .DS_Store files since they gum up the igpublisher....
echo "================================================================="
echo === run definitions maker with optional source directory name as first argument ===
echo === create ig.json and ig.xml in $PWD and ../$SOURCE ===
echo "================================================================="
find . -name '.DS_Store' -type f -delete
python3.5 ${path3}definitions.py

### Replace ig.json "Any" object from template with the one below to speed up the build.

-  All the templates except format.html and ex.html have been disabled
-  The "Any" will suppress, the xml and ttl renderings.

In [2]:
new_any = {
      "xml": False,
      "xml-html": False,
      "template-base": "ex.html",
      "swagger": "true",
      "template-format": "format.html",
      "ttl": False,
      "ttl-html": False
    }

p = Path(r'/Users/ehaas/Documents/FHIR/ArgoR4Validator/ig.json')
ig_dict = loads(p.read_text())
ig_dict['defaults']['Any'] = new_any
p.write_text(dumps(ig_dict,indent=4))

41734

### Run the ig publisher locally ( until can run and see on github )

1. run ig publisher to validate examples **it will take a couple of minutes to run** ( until I can use a more stripped down template )

In [ ]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/ArgoR4Validator/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
java -jar ${path1} -ig ig.json

### Disabled code .... Get qa.html output and display
(or alternatively go here: consider using autopublish to share)

### Commit to GitHub 
- for now can see the validation of examples here: https://healthedata1.github.io/USCoreR4Validator/all-examples.html
- In future Will trigger autobuilder and a CI build

In [ ]:
!git -C /Users/ehaas/Documents/FHIR/ArgoR4Validator add .
!git -C /Users/ehaas/Documents/FHIR/ArgoR4Validator commit -m "added new examples to validate"
!git -C /Users/ehaas/Documents/FHIR/ArgoR4Validator push